In [18]:

import os
import shutil
import cv2
import numpy as np
from keras.models import load_model
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.models import Sequential
from keras.layers import Dropout
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split


# Video lesen
video_path = "Vehicle Detection Junktion 1.mp4"
cap = cv2.VideoCapture(video_path)

# Ordner erstellen, um Frames zu speichern
if not os.path.exists('frames'):
    os.makedirs('frames')

# Zähler initialisieren
count = 0

# Output-Ordner erstellen, falls er noch nicht existiert
output_folder = "output_folder"
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Frames extrahieren
while True:
    # Frame lesen
    ret, frame = cap.read()

    # Überprüfen, ob Frame gelesen wurde
    if ret:
        # Wenn Zähler durch 25 teilbar, dann Frame speichern
        if count % 25 == 0:
            # Frame speichern
            frame_path = os.path.join('frames', f"frame{count}.jpg")
            cv2.imwrite(frame_path, frame)

        # Zähler erhöhen
        count += 1

    # Wenn kein Frame gelesen wurde, dann Schleife beenden
    else:
        break

# Zähler für erkannte Fahrzeuge initialisieren
vehicle_count = 0

# Modell initialisieren oder aus Datei laden
model = load_model('vehicle_detection_model.h5')

# Frames extrahieren
frames_folder = "frames"
for frame_name in os.listdir(frames_folder):
    frame_path = os.path.join(frames_folder, frame_name)

    # Bild lesen und skalieren
    img = cv2.imread(frame_path)
    img = cv2.resize(img, (64, 64))
    img_width, img_height = 64, 64

    # Vorhersage auf dem Bild ausführen
    pred = model.predict(np.array([img]))
    pred_class = int(round(pred[0][0]))

    # Wenn ein Fahrzeug erkannt wurde, erhöhen Sie den Zähler und speichern Sie das Bild im Output-Ordner
    if pred_class == 0:
        vehicle_count += 1
        output_path = os.path.join(output_folder, frame_name)
        cv2.imwrite(output_path, img)

print("Anzahl der erkannten Fahrzeuge:", vehicle_count)

# Initialisierung der Variablen für die Anzahl der LKWs, Busse, PKWs, Lieferfahrzeuge, Krafträder und Fahrräder
num_lkw = 0
num_bus = 0
num_pkw = 0
num_liefer = 0
num_krad = 0
num_rad = 0

# Schleife für die Klassifizierung jedes Fahrzeugs
for frame_name in os.listdir(output_folder):
    frame_path = os.path.join(output_folder, frame_name)

    # Bild lesen und skalieren
    img = cv2.imread(frame_path)
    img = cv2.resize(img, (64, 64))

    # Vorhersage auf dem Bild ausführen
    pred = model.predict(np.array([img]))
    pred_class = int(round(pred[0][0]))

    # Klassifizierung des Fahrzeugs basierend auf der Vorhersage
    if pred_class == 1:
        num_lkw += 1
    elif pred_class == 2:
        num_bus += 1
    elif pred_class == 3:
        num_pkw += 1
    elif pred_class == 4:
        num_liefer += 1
    elif pred_class == 5:
        num_krad += 1
    elif pred_class == 6:
        num_rad += 1
    else:
        print("Fahrzeugtyp unbekannt")

# Ausgabe der Anzahl jedes Fahrzeugtyps
print("Anzahl der Fahrräder:", num_rad)
print("Anzahl der Motorräder:", num_krad)
print("Anzahl der PKWs:", num_pkw)
print("Anzahl der Lieferfahrzeuge:", num_liefer)
print("Anzahl der LKWs:", num_lkw)
print("Anzahl der Busse:", num_bus)



#
# Funktion zum Erstellen des Modells
def create_model(optimizer='adam', num_neurons=128, dropout_rate=0.2):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(num_neurons, activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy',
                  optimizer=optimizer, metrics=['accuracy'])
    return model


# Modell als Sequential-Objekt
model = create_model()

img_width, img_height = 64, 64
# Trennen von Trainings- und Testdaten
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

# Optimierung der Hyperparameter
# Definieren der Hyperparameter
batch_size = [16, 32, 64]
epochs = [10, 20, 50]
param_grid = dict(batch_size=batch_size, epochs=epochs)

# Hyperparameter-Grid
param_grid = {
    'optimizer': ['adam', 'sgd'],
    'num_neurons': [64, 128, 256],
    'dropout_rate': [0.1, 0.2, 0.3]
}

# Grid-Suche
grid = GridSearchCV(estimator=model, param_grid=param_grid,
                    cv=3, scoring='accuracy')
grid_result = grid.fit(X_train, y_train)

# Beste Hyperparameter und Bewertung
print("Beste Hyperparameter: ", grid_result.best_params_)



# vorsicht ergänzt noch nicht ersetzt

# Funktion zum Erstellen des Keras-Modells

def create_model(optimizer='adam', num_neurons=64, dropout_rate=0.2):
    model = Sequential()
    model.add(Conv2D(num_neurons, (3, 3), input_shape=(
        img_width, img_height, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(num_neurons, activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy',
                  optimizer=optimizer, metrics=['accuracy'])
    return model


# Wrap Keras-Modell in einen scikit-learn-Schätzer
model = KerasClassifier(build_fn=create_model,
                        epochs=10, batch_size=32, verbose=0)

# Definieren der Hyperparameter
param_grid = {'optimizer': ['adam', 'sgd'], 'num_neurons': [
    32, 64, 128], 'dropout_rate': [0.2, 0.4, 0.6]}

# Grid-Suche
grid = GridSearchCV(estimator=model, param_grid=param_grid,
                    cv=3, scoring='accuracy')
grid_result = grid.fit(X_train, y_train)

# Beste Hyperparameter und Bewertung
print("Beste Hyperparameter: ", grid_result.best_params_)
print("Beste Bewertung: ", grid_result.best_score_)


1/1 [==============================] - 0s 15ms/step
Anzahl der erkannten Fahrzeuge: 0
Anzahl der Fahrräder: 0
Anzahl der Motorräder: 0
Anzahl der PKWs: 0
Anzahl der Lieferfahrzeuge: 0
Anzahl der LKWs: 0
Anzahl der Busse: 0


TypeError: Cannot clone object '<keras.engine.sequential.Sequential object at 0x00000257BAC587C0>' (type <class 'keras.engine.sequential.Sequential'>): it does not seem to be a scikit-learn estimator as it does not implement a 'get_params' method.